In [1]:
# Change the current working directory to the project root
%cd ..

/home/mahoraga/master/projects/multi-agent-system


In [2]:
from graph.node_analyst import node_analyst
from graph.node_critic import node_critic
from graph.node_presenter import node_presenter
from graph.node_researcher import node_researcher
from graph.node_scout import node_scout
from graph.router_scout import router_scout
from graph.state import AgentState
from langgraph.graph import END, StateGraph
from dotenv import load_dotenv
import os
from rich.markdown import Markdown
load_dotenv()

True

In [3]:
workflow = StateGraph(AgentState)
workflow.add_node("analyst", node_analyst)
workflow.add_node("scout", node_scout)
workflow.add_node("researcher", node_researcher)
workflow.add_node("critic", node_critic)
workflow.add_node("presenter", node_presenter)
workflow.set_entry_point("analyst")
workflow.add_edge("analyst", "scout")
workflow.add_edge("researcher", "critic")
workflow.add_edge("critic", "presenter")
workflow.add_edge("presenter", END)
workflow.add_conditional_edges(
    "scout",
    router_scout,
    { "researcher": "researcher", "presenter": "presenter" }
)
app = workflow.compile()

In [4]:
def save_graph(graph, xray=False, save_path="./figs/agent_graph.png"):
    try:
        graph_png = graph.get_graph(xray=xray).draw_mermaid_png()
        with open(save_path, "wb") as f:
            f.write(graph_png)
    except Exception as e:
        import nest_asyncio
        nest_asyncio.apply()
        from langchain_core.runnables.graph import MermaidDrawMethod
        graph_png = graph.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.PYPPETEER)
        with open(save_path, "wb") as f:
            f.write(graph_png)

In [5]:
save_graph(app, xray=True, save_path="./figs/agent_graph_xray.png")

In [6]:
initial_state = {
    "summary": "Federated Learning is a powerful machine learning paradigm that enables collaborative training across devices. However, its distributed nature presents problems of systemic and statistical heterogeneity that generate training instability, thus requiring adaptive and dynamic solutions. Therefore, this paper presents the community with a new way to address these problems using language model agents. We demonstrate the reasons why LM agents should be used in FL, their advantages, and research opportunities. We also conduct two experiments to demonstrate the potential of a simple implementation and how it can be enhanced through sophisticated agent techniques.",
    "keywords": "",
    "search_tries": 0,
    "raw_vanues": "",
    "enriched_venues": "",
    "selected_venues": "",
    "final_decision": "",

}
result = app.invoke(initial_state)


Node Analyst activated
Node Scout activated
Searching conferences with keyword: Federated Learning
- Federated Learning
- Language Model Agents
- Collaborative Training
Router Scout activated
Decision: Max search tries reached. Send to Node Presenter
Node Presenter activated
No venues were selected.
Final decision generated.


In [9]:
Markdown(result['final_decision'])

First, I need to make sure the message is polite and clear. The user is likely a researcher or someone involved in 
AI, so they might be looking for conferences related to federated learning, language model agents, or collaborative
training. Since no conferences were found, I should explain that and maybe suggest alternative ways to find more   
information.                                                                                                       

I should start with a friendly opening, like "Dear [Name]" or "Hi [Name]". Then state that no relevant conferences 
were found. It's important to be concise but thorough. Maybe mention that the keywords were used effectively and   
offer further assistance. Also, check if there's any specific conference they're interested in, but since they     
didn't find any, I should keep it general.                                                                         


Wait, the user might not know the exact conferences, so it's better to keep it general. Avoid any technical jargon.
Make sure the tone is professional yet approachable. Let me structure it: greet, state the search results, explain 
the lack of conferences, offer help, and close politely. Double-check for any errors in the keywords and ensure the
message is free of any markdown.                                                                                   

Dear [Name],                                                                                                       

After conducting a comprehensive search using the keywords Federated Learning, Language Model Agents, and          
Collaborative Training, we found no relevant conferences related to these topics. Could you please let me know if  
there are any specific conferences you're interested in, and I’d be happy to assist further?                       

Best regards,                                                                                                      
[Your Name]                                                                                                        
[Your Position]                                                                                                    
[Your Organization]                                                                                                

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Note: If you have additional keywords or need help refining the search, feel free to provide them!

In [10]:
result['keywords']

['Federated Learning', 'Language Model Agents', 'Collaborative Training']